In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup
import requests
import time, os

In [2]:
chromedriver = "/Applications/chromedriver" # path to the chromedriver executable
os.environ["webdriver.chrome.driver"] = chromedriver

In [193]:
url = "https://www.rover.com/search/?alternate_results=true&override_check=true&accepts_only_one_client=false&apse=false&bathing_grooming=false&cat_care=false&centerlat=41.804000&centerlng=-87.707000&dogs_allowed_on_bed=false&dogs_allowed_on_furniture=false&frequency=onetime&fulltime_availability=true&giant_dogs=false&has_fenced_yard=false&has_house=false&has_no_children=false&is_premier=false&knows_first_aid=false&large_dogs=false&location=60632&location_accuracy=5161&maxprice=150&medium_dogs=false&minprice=0&no_caged_pets=false&no_cats=false&no_children_0_5=false&no_children_6_12=false&non_smoking=false&page=1&person_does_not_have_dogs=false&pet=&petsitusa=false&pet_type=dog&puppy=false&service_type=dog-walking&small_dogs=false&volunteer_donor=false&search_score_debug=false&injected_medication=false&special_needs=false&oral_medication=false&more_than_one_client=false&uncrated_dogs=false&unspayed_females=false&non_neutered_males=false&females_in_heat=false&unactivated_provider=false&premier_matching=false&premier_or_rover_match=false&is_member_of_sitter_to_sitter=false&is_member_of_sitter_to_sitter_plus=false&location_type=geoip&midday_availability=true"
driver = webdriver.Chrome(chromedriver)
driver.get(url)
time.sleep(0.5)

In [194]:
driver.find_element_by_xpath('//*[@id="base-content"]/div[1]/div/div/div[2]/div[3]/button').click()
location_box = driver.find_element_by_xpath('//*[@id="base-content"]/div[1]/div/div/div[2]/div[3]/button').click()

In [195]:
profile_links = []
names = []
locations = []

for page in range(50):
    soup = BeautifulSoup(driver.page_source)
    #getting profile links
    result_cards = soup.find_all('a', class_="SearchResultCard__MemberProfileAnchor-sc-69cmvm-0 jmSVjX")
    profile_links.append([card.get('href') for card in result_cards])
    #getting names
    name_cards = soup.find_all('a', class_='NameRow__NameLink-w16h4v-2 ddRlnv')
    names.append([card.text for card in name_cards])
    #getting location info
    info_cols = soup.find_all('span', class_="InfoColumn__Location-sc-9o301b-2 gjcFLl")
    locations.append([card.text for card in info_cols])
    time.sleep(0.5)
    #click next page
    try:
        driver.find_element_by_xpath('//*[@id="base-content"]/div[2]/div/ul/li[6]').click()
    except:
        pass
    try:
        driver.find_element_by_xpath('//*[@id="base-content"]/div[2]/div/ul/li[5]').click()
    except:
        pass
    time.sleep(0.5)

In [197]:
len(profile_links)

50

In [201]:
flat_profile_links = [link for sublist in profile_links for link in sublist]
flat_names = [name for sublist in names for name in sublist]
flat_locations = [location for sublist in locations for location in sublist]

In [210]:
df = pd.read_csv('sitters.csv')
df.head()

,Name,Location,Link
0,Christian,"Lower West Side, Chicago, IL, 60608",https://www.rover.com/members/christian-b-id-l...
1,Monica & Calvin,"Lower West Side, Chicago, IL, 60608",https://www.rover.com/members/monica-calvin-m-...
2,Alyssa,"New City, Chicago, IL, 60609",https://www.rover.com/members/alyssa-l-pet-lov...
3,Molly,"Lower West Side, Chicago, IL, 60608",https://www.rover.com/members/molly-c-247-dog-...
4,Annie,"Armour Square, Chicago, IL, 60616",https://www.rover.com/members/annie-x-your-1st...


In [215]:
link = flat_profile_links[0]
driver = webdriver.Chrome(chromedriver)
driver.get(link)

In [275]:
soup = BeautifulSoup(driver.page_source, 'lxml')
service_card = soup.find('div', class_='services-card')

In [276]:
services = []
service_list = service_card.find_all('h3', class_='h5 margin-v-x0')
for service in service_list:
    services.append(service.text.strip())

In [295]:
prices = []
price_list = service_card.find_all('div', class_='h2 margin-v-x0')
for price in price_list:
    prices.append(int(price.text.strip().replace('$', '')))

In [306]:
name_card = soup.find('h1', class_='provider-name media-heading nomargin')
name = name_card.text.strip()

'Christian B.'

In [311]:
def get_name(name_card):
    return name_card.text.strip()

#get price per service in form of list of tuples
def get_service_prices(service_card):
    service_prices = []
    
    service_list = service_card.find_all('h3', class_='h5 margin-v-x0')
    price_list = service_card.find_all('div', class_='h2 margin-v-x0')
    
    for service, price in zip(service_list, price_list):
        service_str = service.text.strip()
        price_int = int(price.text.strip().replace('$', ''))
        service_prices.append((service_str, price_int))
    
    return service_prices
     
#convert list of tuples into dictionary 
def Convert(tup, di): 
    di = dict(tup) 
    return di  

In [314]:
import json
sitter_service_prices = []

for link in flat_profile_links:
    driver = webdriver.Chrome(chromedriver)
    driver.get(link)
    soup = BeautifulSoup(driver.page_source, 'lxml')
    
    #name
    name_card = soup.find('h1', class_='provider-name media-heading nomargin')
    name = get_name(name_card)
    
    #service_prices dict
    service_card = soup.find('div', class_='services-card')
    service_prices_list = get_service_prices(service_card)
    dictionary = {}
    service_prices = Convert(service_prices_list, dictionary)
    
    profile = {'Name': name, 'Services': service_prices}
    sitter_service_prices.append(profile)
    
    driver.close()
    
with open('service_prices.json', 'w') as fout:
    json.dump(sitter_service_prices, fout)

In [343]:
df_1 = pd.read_json('service_prices.json')
service_df = pd.json_normalize(df_1.Services)

In [319]:
import csv
profile_links = []
with open('sitters.csv', 'r') as csv_file:
    csv_reader = csv.DictReader(csv_file, delimiter=',')
    for row in csv_reader:
        profile_links.append(row['Link']) 

In [321]:
len(profile_links)

1000

In [337]:
df_1.Name

0            Christian B.
1      Monica & Calvin M.
2               Alyssa L.
3                Molly C.
4                Annie X.
              ...        
995            Rebecca B.
996             Alexis M.
997            Jessica G.
998              saade h.
999            Rebecca R.
Name: Name, Length: 1000, dtype: object

In [344]:
service_df_final = pd.concat([df_1[['Name']], service_df], axis=1)
service_df_final.to_csv('service_prices.csv', index=False)

In [345]:
service_df_final

,Name,House Sitting,Drop-In Visits,Dog Walking,Boarding,Doggy Day Care
0,Christian B.,30.0,16.0,20,NaN,NaN
1,Monica & Calvin M.,NaN,20.0,20,45.0,40.0
2,Alyssa L.,NaN,20.0,20,NaN,NaN
3,Molly C.,NaN,20.0,18,40.0,33.0
4,Annie X.,65.0,25.0,29,NaN,NaN
...,...,...,...,...,...,...
995,Rebecca B.,20.0,10.0,15,NaN,NaN
996,Alexis M.,40.0,20.0,20,NaN,NaN
997,Jessica G.,50.0,20.0,20,NaN,40.0
998,saade h.,65.0,10.0,15,65.0,20.0


In [335]:
# code to save service price data into csv directly from scraping
try:
    housesit_p = profile['Services']['House Sitting']
except:
    houseit_p = None

try:
    dropin_p = profile['Services']['Drop-In Visits']
except:
    dropin_p = None
    
try:
    walk_p = profile['Services']['Dog Walking Visits']
except:
    walk_p = None
    
try:
    boarding_p = profile['Services']['Boarding Visits']
except:
    boarding_p = None
    
try:
    daycare_p = profile['Services']['Doggy Day Care']
except:
    daycare_p = None
    
services = [name, housesit_p, dropin_p, walk_p, boarding_p, daycare_p]

with open('sitter_prices.csv', 'w') as csv_file:
    writer = csv.writer(csv_file)
    writer.writerow(['Name', 'House Sitting', 'Drop-In Visits', 'Dog Walking', 'Boarding', 'Doggy Day Care'])
    for row in rows:
        writer.writerow(row)